<a href="https://colab.research.google.com/github/dykym/games/blob/master/Mancala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import copy

In [0]:
class Board:
  def hid(self, turn, id):
    tid = 0 if turn == 1 else 1
    return tid * (self.NH + 1) + id
  def __init__(self, num_hole):
    self.NH = num_hole
    self.hole = [0 for x in range(self.NH * 2 + 2)]
    # NH, 2*NH+1はgoal
    self.turn = 1 # sente=1, gote=-1
    self.initialize(4)
  def initialize(self, num_stone):
    for t in range(2):
      for i in range(self.NH):
        self.hole[self.hid(t, i)] = num_stone
      self.hole[self.hid(t, self.NH)] = 0
  def what(self):
    ret = "|"
    for t in [1, -1]:
      ret += "".join([str(x) for x in self.hole[self.hid(t, 0):self.hid(t, self.NH)]])
      ret += "|"
      ret += str(self.hole[self.hid(t, self.NH)])
      ret += "|"
    ret += "(SEN)" if self.turn == 1 else "(GO)"
    return ret
  def stones(self, turn, id):
    return self.hole[self.hid(turn, id)]
  def seed(self, turn, id):
    if turn != self.turn:
      raise Exception('invalid turn')
    p = self.hid(turn, id)
    n = self.hole[p]
    self.hole[p] = 0
    while n > 0:
      p = (p + 1) % len(self.hole)
      self.hole[p] += 1
      n -= 1
    cont = ((p == self.NH) or (p == self.NH * 2 + 1)) # 大きい穴で終わったらtrue
    if not cont:
      self.turn *= -1
    return cont
  def sum_stones(self, turn):
    return sum([self.stones(turn, i) for i in range(self.NH)])
  


In [0]:
class Game:
  def __init__(self):
    self.b = Board(6)
    self.b.initialize(4)
    self.WIN = 999
  def gen_all_moves(self, b, turn):
    ret = []
    for i in range(b.NH):
      if b.stones(turn, i) > 0:
        ret.append(i)
    return ret
  def eval_node(self, b, turn):
    my_sum = b.sum_stones(turn)
    if my_sum == 0:
      return self.WIN * turn
    return (-my_sum) * turn
  def max_node(self, b, remain):
    if remain <= 0:
      return (self.eval_node(b, 1), [])
    #if self.eval_node(b) == self.WIN:
    #  return (self.WIN, [])
    ms = self.gen_all_moves(b, 1)
    if len(ms) == 0:
      return (self.WIN, [])
    #print("depth", remain, ms)
    cvals = []
    for m in ms:
      mb = copy.deepcopy(b)
      cont = self.move(mb, m)
      if mb.sum_stones(1) == 0:
        cont = True
      if cont:
        cvals.append((self.max_node(mb, remain), m))
      else:
        cvals.append((self.min_node(mb, remain - 1), m))
    bestchildvs, bestmove = max(cvals, key=lambda x : x[0][0])
    bestval, bestcmoves = bestchildvs
    retmoves = ["s" + str(bestmove)] + bestcmoves
    return (bestval, retmoves)
  def min_node(self, b, remain):
    if remain <= 0:
      return (self.eval_node(b, -1), [])
    #if self.eval_node(b) == self.WIN:
    #  return (self.WIN, [])
    ms = self.gen_all_moves(b, -1)
    if len(ms) == 0:
      return (-self.WIN, [])
    cvals = []
    for m in ms:
      mb = copy.deepcopy(b)
      cont = self.move(mb, m)
      if mb.sum_stones(-1) == 0:
        cont = True
      if cont:
        cvals.append((self.min_node(mb, remain), m))
      else:
        cvals.append((self.max_node(mb, remain - 1), m))
    bestchildvs, bestmove = min(cvals, key=lambda x : x[0][0])
    bestval, bestcmoves = bestchildvs
    retmoves = ["g" + str(bestmove)] + bestcmoves
    return (bestval, retmoves)
  def move(self, b, mv):
    return b.seed(b.turn, mv)
  def search(self, b, depth):
    turn = b.turn
    if turn == 1:
      r = self.max_node(b, depth)
      turn_s = 's'
    else:
      r = self.min_node(b, depth)
      turn_s = 'g'
    print("result: ", )
    print(r)
    moves = [r[1][0]]
    for m in r[1][1:]:
      if m[0] != turn_s:
        break
      moves.append(m)
    for m in moves:
      mv = int(m[1:])
      b.seed(turn, mv)


  def cui(self):
    b = Board(3)
    b.initialize(2)
    while True:
      print(b.what())
      c = input()
      if c[0] == 'm':
        b.seed(b.turn, int(c[2:]))
      elif c[0] == 's':
        elms = c.split(' ')
        if len(elms) != 2:
          depth = 10
        else:
          depth = int(elms[1])
        self.search(b, depth)
      elif c[0] == 'i':
        elms = c.split(' ')
        b = Board(int(elms[1]))
        b.initialize(int(elms[2]))
      elif c == 'q':
        break
      elif c == 'h':
        print("m num")
        print("q: quit")


  def test(self):
    b = Board(3)
    b.initialize(2)
    print(b.what())
    r = self.max_node(b, 20)
    print(r)
   

In [0]:
class Mancala:
  def main(self):
    Game().cui()

In [24]:
Game().test()

|222|0|222|0|(SEN)
(-1, ['s0', 'g1', 'g2', 's0', 'g0', 's1', 'g2', 's0', 'g0', 's2', 'g0', 's1', 'g1', 's2', 'g0', 's0', 'g2', 's0', 'g1', 's1', 'g0'])


In [26]:
Mancala().main()

|222|0|222|0|(SEN)
s 3
result: 
(6, ['s1', 's2', 'g1', 's0', 's1'])
|200|2|332|0|(GO)
s 3
result: 
(-4, ['g1', 's0', 's2', 's1', 'g0', 'g2'])
|300|2|303|1|(SEN)
s 3
result: 
(3, ['s0', 's1', 'g0', 'g2', 's2'])
|002|3|303|1|(GO)
s 3
result: 
(-3, ['g2', 's2', 'g0'])
|112|3|300|2|(SEN)
s 3
result: 
(999, ['s0', 'g0', 'g1', 's1', 's2'])
|022|3|300|2|(GO)
s 3
result: 
(999, ['g0', 'g1', 's1', 's2'])
|022|3|002|3|(SEN)
s 3
result: 
(999, ['s1', 's2'])
|000|5|112|3|(GO)
q
